In [1]:
## Web server Flask

In [ ]:
from flask import Flask, jsonify
import psycopg2
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

def db_connection():
    conn = psycopg2.connect(
        dbname='postgres',
        user='postgres',
        password='.s41352491.',
        host='localhost'
    )
    return conn

@app.route('/comunes', methods=['GET'])
def getComunes():
    conn = db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('''
        SELECT DISTINCT ON (apd.nome) apd.*, 
               COALESCE(ST_AsGeoJSON(c.geom), '{}') as geom 
        FROM public.api_data_new apd 
        LEFT JOIN public.comuni c ON c.name = apd.nome;
    ''')
    comunes = cursor.fetchall()
    cursor.close()
    conn.close()
    
    # Ensure proper encoding and format
    for comune in comunes:
        if isinstance(comune['geom'], str):
            comune['geom'] = comune['geom'].encode('utf-8')  # Ensure utf-8 encoding

    return jsonify(comunes)

@app.route('/comunes/<int:id>', methods=['GET'])
def getComunesById(id):
    conn = db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('''
        SELECT DISTINCT ON (apd.nome) apd.*, ST_AsGeoJSON(c.geom, '{}') as geom 
        FROM public.api_data_new apd 
        LEFT JOIN public.comuni c ON c.name = apd.nome 
        WHERE apd.uid = %s;
    ''', (id,))
    comune = cursor.fetchone()
    cursor.close()
    conn.close()
    return jsonify(comune)

@app.route('/')
def index():
    return 'Hello, World!'


if __name__ == '__main__':
    import os
    import sys
    if 'ipykernel' in sys.modules:
        from werkzeug.serving import run_simple
        run_simple('localhost', 5000, app)
    else:
        app.run(debug=True)
